In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from mpl_toolkits.mplot3d import Axes3D
from IPython import display

from scipy.spatial.transform import Rotation as R

import math
from copy import copy
import glob

In [ ]:
flist = sorted(glob.glob('../results/subsets/order*'))

In [ ]:
# Show demo 
f = np.random.choice(flist)
run = int(f.split('_')[1][:-4])

ims = np.load('../results/subsets/ims_%d.npy'%run)
states = np.load('../results/subsets/states_%d.npy'%run)
obj_ids = np.load('../results/subsets/order_%d.npy'%run)

for i, im in enumerate(ims):
    if i %10 == 0:
        plt.cla()
        plt.clf()
        plt.subplot(2,2,1)
        plt.imshow(im[0])
        plt.subplot(2,2,2)
        plt.imshow(im[1])
        plt.subplot(2,1,2)
        plt.plot(np.vstack(states)[0:i,:])
        display.clear_output(wait=True)
        display.display(plt.gcf())

In [ ]:
obj_list = []
im_list = []
for i,f in enumerate(flist[0:300]):
    run = int(f.split('_')[1][:-4])

    ims = np.load('../results/subsets/ims_%d.npy'%run)
    obj_ids = np.load('../results/subsets/order_%d.npy'%run)
    
    obj_list.append(obj_ids)
    im_list.append(ims[-1][0])

In [ ]:
obj_idxs = np.ones((len(obj_list),6))*100
seq_len = []
for j,obj_idx in enumerate(obj_list):
    seq_len.append(obj_idx.shape[0])
    obj_idxs[j,0:obj_idx.shape[0]] = obj_idx

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [ ]:
# Set up pytorch dataloader
class Sampler(Dataset):
    
    def __init__(self, ims, actions, seq_lens, K=6):
        
        self.ims = torch.FloatTensor(ims.astype('float'))
        self.actions = torch.FloatTensor(actions.astype('float'))
        self.seq_lens = torch.LongTensor(seq_lens.astype('int'))
        self.K = K
        
        
    def __len__(self):
        
        return self.ims.shape[0]
    
    def __getitem__(self, index):
        
        im = self.ims[index,:,:,:]
        actions = self.actions[index,:]
        seq_len = self.seq_lens[index]
        return im, actions, torch.FloatTensor(np.arange(self.K).astype('float')), seq_len


In [ ]:
dataset = Sampler(np.swapaxes(np.stack(im_list),1,3),obj_idxs,np.array(seq_len),6)

train_dataset,test_dataset = torch.utils.data.random_split(dataset, [200,len(im_list)-200])

In [ ]:
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)
    
class SinkhornNet(nn.Module):

    def __init__(self, latent_dim=16, image_channels=3, K=6, n_samples=5, noise_factor=1.0, temp=1.0, n_iters=20):
        super(SinkhornNet, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=5, stride=2),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=5, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=5, stride=2),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=5, stride=2),
            nn.ReLU(),
            Flatten(),
            nn.Linear(256, latent_dim),
            nn.Dropout(p=0.2),
        )
        
        # Sinkhorn params
        self.latent_dim = latent_dim
        self.K = K
        self.n_samples = n_samples
        self.noise_factor = noise_factor
        self.temp = temp
        self.n_iters = n_iters
        
        self.criterion = nn.MSELoss(reduction='none')
        self.mask_criterion = nn.CrossEntropyLoss()

        self.sinknet = nn.Sequential(
                        nn.Linear(self.latent_dim, K*K))
        
        self.masknet = nn.Sequential(
                        nn.Linear(self.latent_dim, K),nn.Softmax(dim=1))
    
    def permute(self,seq,P):
        
        return torch.matmul(P,seq)
    
    def predict_P(self,im):
        
        latent = self.encoder(im)
        log_alpha = self.sinknet(latent)
        log_alpha = log_alpha.reshape(-1, self.K, self.K)
         
        soft_perms_inf, log_alpha_w_noise = self.gumbel_sinkhorn(log_alpha)
        
        P = self.inv_soft_pers_flattened(soft_perms_inf,self.K)
        return P
    
    def forward(self, seq, im):
        
        latent = self.encoder(im)
        log_alpha = self.sinknet(latent)
        log_alpha = log_alpha.reshape(-1, self.K, self.K)
         
        soft_perms_inf, log_alpha_w_noise = self.gumbel_sinkhorn(log_alpha)
        
        P = self.inv_soft_pers_flattened(soft_perms_inf,self.K)
        
        seq_tiled = seq.repeat(self.n_samples, 1)
        ordered  = self.permute(torch.unsqueeze(seq_tiled,dim=-1),P)

        stopping = self.masknet(latent)
        
        return ordered, stopping
    
    def loss(self, seq, im, seq_gt, seq_len):
        
        seq_pred,stopping_bin = self.forward(seq_gt,im)
        seq_pred = torch.squeeze(seq_pred)

        mask_loss = self.mask_criterion(stopping_bin,seq_len)
        
        mask_idxs = torch.arange(0,self.K).repeat(seq_pred.size(0),1)
        
        seq_len_mask = seq_len.reshape(-1,1).repeat(self.n_samples,self.K)
       
        stopping_mask = (mask_idxs<seq_len_mask).int().float()
       
        recon_loss = self.criterion(seq_pred*stopping_mask,seq.repeat(self.n_samples, 1)*stopping_mask)
        
        recon_loss = (recon_loss.sum(dim=-1)/seq_len.repeat(self.n_samples).int().float()).mean()
        
        return recon_loss, mask_loss, seq_pred,stopping_mask
    
    def inv_soft_pers_flattened(self,soft_perms_inf,n_numbers):
        inv_soft_perms = torch.transpose(soft_perms_inf, 2, 3)
        inv_soft_perms = torch.transpose(inv_soft_perms, 0, 1)

        inv_soft_perms_flat = inv_soft_perms.view(-1, n_numbers, n_numbers)
        return inv_soft_perms_flat
    
    def sample_gumbel(self, shape, eps=1e-20):
        U = torch.rand(shape).float()
        return -torch.log(eps - torch.log(U + eps))
    
    def gumbel_sinkhorn(self,log_alpha):
        
        n = log_alpha.size()[1]
        log_alpha = log_alpha.view(-1, n, n)
        batch_size = log_alpha.size()[0]

        log_alpha_w_noise = log_alpha.repeat(self.n_samples, 1, 1)

        if self.noise_factor == 0:
            noise = 0.0
        else:
            noise = self.sample_gumbel([self.n_samples*batch_size, n, n])*self.noise_factor

        log_alpha_w_noise = log_alpha_w_noise + noise
        log_alpha_w_noise = log_alpha_w_noise / self.temp

        my_log_alpha_w_noise = log_alpha_w_noise.clone()

        sink = self.sinkhorn(my_log_alpha_w_noise)

        sink = sink.view(self.n_samples, batch_size, n, n)
        sink = torch.transpose(sink, 1, 0)
        log_alpha_w_noise = log_alpha_w_noise.view(self.n_samples, batch_size, n, n)
        log_alpha_w_noise = torch.transpose(log_alpha_w_noise, 1, 0)

        return sink, log_alpha_w_noise
    
    def sinkhorn(self,log_alpha, n_iters = 20):
   
        n = log_alpha.size()[1]
        log_alpha = log_alpha.view(-1, n, n)

        for i in range(n_iters):
            # torch.logsumexp(input, dim, keepdim, out=None)
            #Returns the log of summed exponentials of each row of the input tensor in the given dimension dim
            #log_alpha -= (torch.logsumexp(log_alpha, dim=2, keepdim=True)).view(-1, n, 1)
            #log_alpha -= (torch.logsumexp(log_alpha, dim=1, keepdim=True)).view(-1, 1, n)
            #avoid in-place
            log_alpha = log_alpha - (torch.logsumexp(log_alpha, dim=2, keepdim=True)).view(-1, n, 1)
            log_alpha = log_alpha - (torch.logsumexp(log_alpha, dim=1, keepdim=True)).view(-1, 1, n)
        return torch.exp(log_alpha)


In [ ]:
sn = SinkhornNet(latent_dim=128, image_channels=3, K=6)
optimizer = torch.optim.Adam(sn.parameters(), lr=1e-4)
n_epochs = 500
losses = []
losses_m = []

In [ ]:
plt.figure(figsize=(15,6))

for j in range(n_epochs):
    
    batch_losses = []
    batch_losses_m = []
    for im, seq, seq_order,seq_len in train_loader:
    
        loss_r,loss_m, seq_pred,mask = sn.loss(seq, im, seq_order,seq_len)
        
        batch_losses.append(loss_r.item())
        batch_losses_m.append(loss_m.item())
        
        loss = loss_r + loss_m
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
    losses.append(np.mean(batch_losses))
    losses_m.append(np.mean(batch_losses_m))
    
    plt.clf()
    plt.cla()
    plt.subplot(2,3,1)
    plt.plot(losses,alpha=0.5)
    plt.title('Recon loss')
    plt.subplot(2,3,4)
    plt.plot(losses_m,alpha=0.5)
    plt.title('Mask loss')

    plt.subplot(2,3,6)
    P = sn.predict_P(im)
    plt.imshow(P[0,:,:].detach().numpy())
    plt.title('Permutation')
    
    plt.subplot(3,3,2)
    plt.imshow(seq_pred[0,:].detach().numpy().reshape(1,-1))
    plt.title('Re-shuffled seq')
    
    plt.subplot(3,3,5)
    plt.imshow(seq_order[0,:].detach().numpy().reshape(1,-1))
    plt.title('Ordered seq')
    
    plt.subplot(3,3,8)
    plt.imshow(seq[0,:].detach().numpy().reshape(1,-1))
    plt.title('Original seq')
    
    plt.subplot(2,3,3)
    plt.imshow(np.swapaxes(im[0,:,:,:].detach().numpy(),0,2))
    
        
    display.clear_output(wait=True)
    display.display(plt.gcf())
    

In [ ]:
# Action sequencing using visual permutations 

# Exps -> compare to classification baseline
# Exps -> generalisation with respect to unseen configs?
# Exps -> 

In [ ]:
from pyrep import PyRep
from pyrep.robots.arms.panda import Panda
from pyrep.robots.end_effectors.panda_gripper import PandaGripper
import numpy as np
from matplotlib import pyplot as plt

from mpl_toolkits.mplot3d import Axes3D
from IPython import display

from pyrep.objects.shape import Shape
from pyrep.const import PrimitiveShape

from scipy.spatial.transform import Rotation as R

import math
from copy import copy

In [ ]:
pr = PyRep()
pr.launch('../assets/scene_panda.ttt', headless=False)

In [ ]:
agent = Panda()
gripper = PandaGripper()

In [ ]:
home_pos = agent.get_tip().get_position()
home_orient = agent.get_tip().get_orientation()

In [ ]:
def grasp(grip=False):
    if grip:
        pos = 0.1
    else:
        pos = 0.9
    actuated = False
    ims = []
    states = []
    while not actuated:
        actuated = gripper.actuate(pos,0.1)
        im0,im1 = get_image()
        ims.append((im0,im1))
        states.append(agent.get_tip().get_pose())
    return ims,states

In [ ]:
def move_above_object(object_name='',offset=0.05):
    pos = agent.get_object(object_name).get_position()
    pos[2] = pos[2] + offset
    orient = [-np.pi,0,np.pi/2]

    path = agent.get_path(position=pos,euler=orient)

    done = False
    ims = []
    states = []
    while not done:
        done = path.step()
        im0,im1 = get_image()
        ims.append((im0,im1))
        states.append(agent.get_tip().get_pose())
    return ims,states

In [ ]:
def get_image():
    cam = agent.get_object('Vision_sensor_front')
    im0 = cam.capture_rgb()
    cam1 = agent.get_object('Vision_sensor')
    im1 = cam1.capture_rgb()
    pr.step()
    return im0, im1

In [ ]:
sn.eval()
sn.noise_factor = 0.0
sn.n_samples = 1
sn.n_iters = 100

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
from scipy.stats import kendalltau

In [ ]:
tau_list = []
Acc = []
for im,seq,seq_ordered,seq_len in test_loader:
    P = sn.predict_P(im)
    order,stop = sn(seq_ordered,im)
    Acc.append(np.argmax(stop.detach().numpy(),1)==seq_len.numpy())
#     obj_ids = np.round(np.matmul(np.linalg.inv(P[0,:,:].detach().numpy()),seq_ordered[0,:].numpy())).astype(int)
    obj_ids = np.argmax(P[0,:,:].detach().numpy(),1)
    tau, _ = kendalltau(obj_ids[0:seq_len.numpy()[0]], seq[0,0:seq_len.numpy()[0]].numpy())
    tau_list.append(tau)


In [ ]:
print('Stopping accuracy:',np.sum(Acc)/len(Acc))

In [ ]:
plt.plot(tau_list)
plt.show()

In [ ]:
tl = iter(train_loader)

In [ ]:
im,seq,seq_ordered,seq_len = next(tl)
P = sn.predict_P(im)
_,stop_logit = sn(seq_ordered,im)

In [ ]:
#obj_ids = np.round(np.matmul(np.linalg.inv(P[0,:,:].detach().numpy()),seq_ordered[0,:].numpy())).astype(int)
# from scipy.optimize import linear_sum_assignment
# _,obj_ids = linear_sum_assignment(1.0-P[0,:,:].detach().numpy())

stop_bin = np.argmax(stop_logit.detach().numpy(),-1)

obj_ids = np.argmax(P[0,:,:].detach().numpy(),1)[0:stop_bin[0]]


In [ ]:
plt.subplot(1,3,1)
plt.imshow(P[0,:,:].detach().numpy())
plt.subplot(1,3,2)
plt.imshow(np.linalg.inv(P[0,:,:].detach().numpy()))
plt.subplot(1,3,3)
plt.imshow(np.swapaxes(im[0,:,:,:].detach().numpy(),0,2))
plt.show()


In [ ]:
pr.stop()

In [ ]:
pr.start()

grasp(grip=False)
gripper.release()
ims = []
states = []
for j in range(1,len(obj_ids)):
    object_name = 'Cuboid%d'%obj_ids[j]
    i,s = move_above_object(object_name,offset=0.08)
    ims = ims + i
    states = states + s
    i,s = move_above_object(object_name,offset=0)
    ims = ims + i
    states = states + s
    i,s = grasp(grip=True)
    ims = ims + i
    states = states + s
    gripper.grasp(agent.get_object(object_name))
    i,s = move_above_object(object_name,offset=0.08)
    ims = ims + i
    states = states + s
    object_name = 'Cuboid%d'%obj_ids[j-1]
    i,s = move_above_object(object_name,offset=0.15)
    ims = ims + i
    states = states + s
    i,s = move_above_object(object_name,offset=0.05)
    ims = ims + i
    states = states + s
    i,s = grasp(grip=False)
    ims = ims + i
    states = states + s
    gripper.release()
    i,s = move_above_object(object_name,offset=0.2)
    ims = ims + i
    states = states + s
    #clear_view()
    
    plt.cla()
    plt.clf()
    plt.subplot(1,2,1)
    plt.imshow(ims[-1][0])
    plt.subplot(1,2,2)
    plt.imshow(ims[-1][1])
    display.clear_output(wait=True)
    display.display(plt.gcf())

pr.stop()

In [ ]:
f = np.random.choice(glob.glob('./results/order*'))
run = int(f.split('_')[1][:-4])

im = np.load('./results/ims_%d.npy'%run)[-1][0]
obj_ids = np.load('./results/order_%d.npy'%run)

P = sn.predict_P(torch.from_numpy(np.swapaxes(im,0,2).reshape(1,3,64,64)).float())

obj_ids = np.argmax(np.linalg.inv(P[0,:,:].detach().numpy()),1)

plt.subplot(1,2,1)
plt.imshow(im)
plt.subplot(1,2,2)
plt.imshow(P[0,:,:].detach())
plt.show()

In [ ]:
# Test on unseen configuration


pr.start()

grasp(grip=False)
gripper.release()
ims = []
states = []
for j in range(1,6):
    object_name = 'Cuboid%d'%obj_ids[j]
    i,s = move_above_object(object_name,offset=0.08)
    ims = ims + i
    states = states + s
    i,s = move_above_object(object_name,offset=0)
    ims = ims + i
    states = states + s
    i,s = grasp(grip=True)
    ims = ims + i
    states = states + s
    gripper.grasp(agent.get_object(object_name))
    i,s = move_above_object(object_name,offset=0.08)
    ims = ims + i
    states = states + s
    object_name = 'Cuboid%d'%obj_ids[j-1]
    i,s = move_above_object(object_name,offset=0.15)
    ims = ims + i
    states = states + s
    i,s = move_above_object(object_name,offset=0.05)
    ims = ims + i
    states = states + s
    i,s = grasp(grip=False)
    ims = ims + i
    states = states + s
    gripper.release()
    i,s = move_above_object(object_name,offset=0.2)
    ims = ims + i
    states = states + s
    #clear_view()
    
    plt.cla()
    plt.clf()
    plt.subplot(1,2,1)
    plt.imshow(ims[-1][0])
    plt.subplot(1,2,2)
    plt.imshow(ims[-1][1])
    display.clear_output(wait=True)
    display.display(plt.gcf())

pr.stop()

In [ ]:
pr.shutdown()

In [ ]:
# Exps -> generalisation?
# Exps -> multi-seed
# Graphs:
# ---> Kendals tau

# Baselines: 
# ---> Behaviour cloning
# ---> Behaviour cloning + Hungarian algorithm